MLSQL快速入门
----

MLSQL 致力于为数据平台提供 **低门槛的机器学习开发和使用**，允许用户 **以 SQL 的形式** 使用机器学习算法进行 模型训练、预测等功能，并提供统一的自定义算法的开发。

接下来以 **鸢尾花分类为例**（使用逻辑回归算法，根据鸢尾花的花瓣、花萼大小，判断其种类 ），快速了解 MLSQL 的核心用法。  

> 鸢尾花有3种类别：setosa、versicolor、virginica


- Iris setosa
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg/900px-Kosaciec_szczecinkowaty_Iris_setosa.jpg" width = "8%" /> 

- Iris versicolor
<img src="https://upload.wikimedia.org/wikipedia/commons/4/41/Iris_versicolor_3.jpg" width = "10%" /> 

- virginica
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/440px-Iris_virginica.jpg" width = "10%" />  



### 1. 准备样本
基于公开的 [安德森鸢尾花卉数据集](https://zh.wikipedia.org/wiki/%E5%AE%89%E5%BE%B7%E6%A3%AE%E9%B8%A2%E5%B0%BE%E8%8A%B1%E5%8D%89%E6%95%B0%E6%8D%AE%E9%9B%86)创建一个结果表 591_iris_sample_queryset，用作模型训练的样本。

注：请修改该笔记中存在名称冲突的查询结果集、模型名称，例如将 591_iris_sample_queryset 改成 591_iris_sample_queryset100

In [1]:
%%bksql
create table 591_iris_sample_queryset as select sepal_length, sepal_width, petal_width, petal_length, species 
FROM `591_iris_sample`.`hdfs`
WHERE `thedate` = '20201127'
LIMIT 150 

'创建成功'

查询刚创建的查询结果集，在下图中看到模型训练需要的 **特征** 和 **标签**

- 特征：sepal_length（花萼长度）, sepal_width（花萼宽度）, petal_width（花瓣宽度）, petal_length（花瓣长度）
- 标签： species（鸢尾花的属种）

从气泡图可以看出三种花的分布：setosa 位于左下角，versicolor 在中间，virginica 位于右上角。

In [2]:
%%bksql
select sepal_length, sepal_width, petal_width, petal_length, species
FROM 591_iris_sample_queryset
limit 150

页面最多展示100条数据，请将结果集赋值给变量进行处理，示例：result = _


### 2. 数据处理与特征工程：将字符串转为 double （可选）
由于该样本中的标签列（species）的字段类型是 `字符串`，需要使用`特征转换`将其转换为模型训练需要的 `double` 类型；

> 在 MLSQL 中，有些特征处理的算法需要创建 model，模型发布时包含该特征转换的 model。

使用 string_indexer 特征处理算法（参考平台帮助文档），将 species 列的值（字符串）转换为 double 类型的值，并命名为 `species_num`。

接下来使用 train model 语法，构建特征转换的 model。

In [3]:
%%mlsql
train model 591_transform_string_to_double
options(algorithm='string_indexer',
        input_col='species', output_col='species_num')
from 591_iris_sample_queryset

{'bk_status': [{'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'parse', 'description': 'SQL解析', 'stage_seq': 1}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'submit', 'description': '提交任务', 'stage_seq': 2}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'execute', 'description': '任务执行', 'stage_seq': 3}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:15', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 2}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:15', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 5}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:15', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 8}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:15', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 11}]}


...

{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:15', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 17}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:15', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 20}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:15', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 24}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:15', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'success', 'stage_start': '2020-12-01 09:49:16', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 24}], 'bk_release': 'false'}


'执行成功'

接下来使用 run model 语法，完成特征转换，并写入到查询结果集中；

注：在后续迭代中，可以在分类/回归、推荐、聚类等算法模型训练的同时一并完成特征转换。

In [4]:
%%mlsql

create table 591_iris_sample_qs_double 
    as run 591_transform_string_to_double(input_col=species) as species_num, 
    sepal_length, sepal_width, petal_width, petal_length
    from 591_iris_sample_queryset;


{'bk_status': [{'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'parse', 'description': 'SQL解析', 'stage_seq': 1}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'submit', 'description': '提交任务', 'stage_seq': 2}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'execute', 'description': '任务执行', 'stage_seq': 3}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:04', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 3}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'execute', 'description': '任务执行', 'stage_seq': 3}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:04', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 3}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 3}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:04', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 3}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 6}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:04', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 3}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 9}]}


...

{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:04', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 3}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 15}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:04', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 3}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 19}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:04', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 3}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 22}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:04', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 3}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'success', 'stage_start': '2020-12-01 09:57:07', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 22}], 'bk_release': 'true'}


'执行成功'

查询特征转换后的查询结果集，可以看到，species 列的字符串(`versicolor, virginica, setosa` )已经转换为 `0.0, 1.0, 2.0`

In [5]:
%%bksql

select * from 591_iris_sample_qs_double limit 150;

页面最多展示100条数据，请将结果集赋值给变量进行处理，示例：result = _


### 3. 模型训练（算法：逻辑回归）
预测花的分类属于分类问题，可以使用 `逻辑回归` 算法进行监督学习，预测某个值是否属于多个类别之一。

- 输入特征：sepal_length（花萼长度）, sepal_width（花萼宽度）, petal_width（花瓣宽度）, petal_length（花瓣长度）
- 标签：species_num（花的属种）


使用 `train model` 语法，并选择算法 `algorithm='logistic_regression'`, 会创建一个 model，并使用上面特征转换后的查询结果集进行模型训练。


In [6]:
%%mlsql

train model 591_iris_species_predict
   options(
       algorithm='logistic_regression', 
       features_col=<sepal_length, sepal_width, petal_width, petal_length>, 
       label_col=species_num, max_iter=10 ) 
from 591_iris_sample_qs_double

{'bk_status': [{'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'parse', 'description': 'SQL解析', 'stage_seq': 1}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'submit', 'description': '提交任务', 'stage_seq': 2}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'execute', 'description': '任务执行', 'stage_seq': 3}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:10', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 2}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:10', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 5}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:10', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 8}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:10', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 11}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:10', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 14}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:10', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 17}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:10', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 20}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:10', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 1}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'success', 'stage_start': '2020-12-01 17:06:11', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 20}], 'bk_release': 'false'}


'执行成功'

### 4. 模型评估（开发中，暂未开放）
> 内置的评估函数还在开发中，可以先自行编写评估函数；

用验证集的数据对已训练的模型进行评估。如果评估得分符合预期，可以做模型预测或发布；

### 5. 模型预测
使用 run model 语法直接在笔记中对数据进行预测。



In [7]:
%%mlsql
create table 591_iris_predict_result as
    run 591_iris_species_predict(features_col=<sepal_length, sepal_width, petal_width, petal_length>) as predict_label,
    sepal_length, sepal_width, petal_width, petal_length
from 591_iris_sample_qs_double

{'bk_status': [{'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'parse', 'description': 'SQL解析', 'stage_seq': 1}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'submit', 'description': '提交任务', 'stage_seq': 2}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'execute', 'description': '任务执行', 'stage_seq': 3}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'running', 'stage_start': '2020-12-01 17:07:02', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 3}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'submit', 'description': '提交任务', 'stage_seq': 2}, {'stage_status': 'pending', 'time_taken': 0, 'stage_type': 'execute', 'description': '任务执行', 'stage_seq': 3}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:02', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 4}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 2}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:02', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 4}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 5}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:02', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 4}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 8}]}


...

{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:02', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 4}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 14}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:02', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 4}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'running', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 18}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:02', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 4}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 21}]}


{'bk_status': [{'description': 'SQL解析', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:02', 'stage_type': 'parse', 'stage_seq': 1, 'time_taken': 4}, {'description': '提交任务', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'submit', 'stage_seq': 2, 'time_taken': 1}, {'description': '任务执行', 'stage_status': 'success', 'stage_start': '2020-12-01 17:07:06', 'stage_type': 'execute', 'stage_seq': 3, 'time_taken': 21}], 'bk_release': 'true'}


'执行成功'

查看预测结果，可以看到 predict_result 列为模型预测的结果

In [8]:
%%bksql
select sepal_length, sepal_width,  petal_length, petal_width, 
    CASE WHEN predict_label=0.0 THEN 'versicolor'
    WHEN predict_label=1.0 THEN 'virginica'
    WHEN predict_label=2.0 THEN 'setosa' 
    ELSE '--' END AS predict_result
from 591_iris_predict_result;

### 6. 模型发布（可选）

如果期望模型被周期性调度，可以对模型预测结果进行发布 。

发布后的模型在平台 **我的模型** 页面可以找到，可以作为数据开发的一个处理节点，被数据开发任务周期性调度。